In [38]:
# pip install transformers

In [39]:
# !pip install datasets

In [40]:
import os
from datasets import Dataset
from transformers import BertTokenizer
import pandas as pd

In [43]:

from transformers import AutoModel, AutoConfig

# Path to the directory containing config.json and model.safetensors
model_directory = '../Models/With_Nationality/model/'

# Load the configuration
config = AutoConfig.from_pretrained(model_directory)

# Load the model using the configuration
model_NN = AutoModel.from_pretrained(model_directory, config=config, use_safetensors=True)



Some weights of BertModel were not initialized from the model checkpoint at ../Models/With_Nationality/model/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')

In [29]:
file_path = '../data/data.csv'
df = pd.read_csv(file_path)

In [30]:
columns_to_drop_without_nationality = ['USER_NUMBER', 'file_name','begin','end','Gender','wrong_word','process','main_category','sub_category_1','sub_category_2','Training Center','Nationality','nationality_encoded','correct_word','word_tokens_correct','correct_sentence','word_tokens_wrong']

In [18]:
# tdwn = df.drop(columns=columns_to_drop_without_nationality)

In [45]:
# Correct `id2label` mapping
model_NN.config.id2label = {
    0: "negative",
    # 1: "MainSigns_Consonants",
    # 2: "MainSigns_Vowels",
    # 3: "DiacriticSigns_Tashdid",
    # 4: "DiacriticSigns_Madd",
    # 5: "DiacriticSigns_Tanwin",
    # 6: "Form_Boundary",
    # 7: "Form_AlifLam",
    # 8: "Form_Place",
    # 9: "Form_Dot",
    # 10: "Punctuation"
    1:"posetive"

}

# Correct `label2id` mapping
model_NN.config.label2id = {
    "negative": 0,
    # "MainSigns_Consonants": 1,
    # "MainSigns_Vowels": 2,
    # "DiacriticSigns_Tashdid": 3,
    # "DiacriticSigns_Madd": 4,
    # "DiacriticSigns_Tanwin": 5,
    # "Form_Boundary": 6,
    # "Form_AlifLam": 7,
    # "Form_Place": 8,
    # "Form_Dot": 9,
    # "Punctuation": 10
    "posetive":1
}


In [54]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# # Load the trained model and tokenizer
# model_path = "path_to_your_trained_model"  # Change to the path of your saved model
# tokenizer_path = "path_to_your_tokenizer"  # Change to the path of your saved tokenizer

# tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
# model = AutoModelForTokenClassification.from_pretrained(model_path)

# # Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Input text
# # # Tokenize the input text
# inputs = tokenizer(text, return_tensors="pt", is_split_into_words=False, padding=True, truncation=True)
# input_ids = inputs["input_ids"]
# attention_mask = inputs["attention_mask"]

# # # Get predictions from the model
# model.eval()  # Set the model to evaluation mode
# with torch.no_grad():
#     outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#     logits = outputs[1]

# # # Convert logits to predicted labels
# predictions = torch.argmax(logits, dim=2)

# # # # Map label IDs to actual labels
# id2label = model.config.id2label
# predicted_labels = []
# for label_id in predictions[0]:
#     label_id = label_id.item()
#     predicted_labels.append(id2label.get(label_id, "Unknown Label"))

# # # # Post-process the predictions
# tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
# print("Tokens:", tokens)
# print("Predicted Labels:", predicted_labels)

# for i in range(0,len(predicted_labels)):
#   if predicted_labels[i]=="False":
#     print("error is in this word :",tokens[i])
# print(outputs['pooler_output'])


# Sample input text
text = "دیوار کلاس سفد است"


# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Forward pass to get outputs
outputs = model_NN(**inputs)

# Get the last hidden state (representation for each token)
last_hidden_state = outputs.last_hidden_state  # shape: (batch_size, seq_len, hidden_size)

# Classification head: Define a dense layer for each token
# Assuming binary classification for each token (you can change this for more classes)
classification_head = torch.nn.Linear(last_hidden_state.shape[2], 1)  # Output size is 1 for binary classification
print(last_hidden_state.shape)
# Forward the last hidden state through the classification head
token_logits = classification_head(last_hidden_state)  # shape: (batch_size, seq_len, 1)
print(token_logits.shape)
# Squeeze the logits to remove the last dimension
token_logits = token_logits.squeeze(-1)  # shape: (batch_size, seq_len)
print(token_logits)
# Sigmoid to convert logits to probabilities (for binary classification)
token_probabilities = torch.sigmoid(token_logits)
print(token_probabilities)
# Get the predicted class for each token (0 or 1)
token_predictions = (token_probabilities > 0.5).int()

# Decode token ids to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Print each token with its predicted label
for token, prediction in zip(tokens, token_predictions[0]):
    print(f"Token: {token}, Predicted class: {prediction.item()}")


torch.Size([1, 7, 768])
torch.Size([1, 7, 1])
tensor([[-0.0402, -0.0712, -0.2514, -0.2748, -0.2308,  0.1243, -0.0346]],
       grad_fn=<SqueezeBackward1>)
tensor([[0.4899, 0.4822, 0.4375, 0.4317, 0.4426, 0.5310, 0.4913]],
       grad_fn=<SigmoidBackward0>)
Token: [CLS], Predicted class: 0
Token: دیوار, Predicted class: 0
Token: کلاس, Predicted class: 0
Token: سف, Predicted class: 0
Token: ##د, Predicted class: 0
Token: است, Predicted class: 1
Token: [SEP], Predicted class: 0
